In [38]:
import requests
import xlrd
import pandas as pd
import numpy as np
import json
import time as time_module 
import geopandas as gpd
import matplotlib.pyplot as plt
import polyline
from shapely.geometry import LineString, MultiLineString
from shapely.ops import linemerge
from collections import defaultdict
import math
from geopy.distance import geodesic
from shapely.ops import nearest_points
import folium
from folium import Element
import geopandas as gpd
import pandas as pd
from pprint import pprint
import os
import re
import glob
from dotenv import load_dotenv
from shapely.geometry import Point


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

# Retrieving api key
load_dotenv("../key.env")
api_key = os.getenv("API_KEY")
TOKEN = os.getenv('ONEMAPTOKEN')

#### Reading in dataset

In [39]:
geospatial_train_path = "../datasets/geospatial_layer/TrainStation_Jul2024/RapidTransitSystemStation.shp"
train_stations = pd.read_excel("../datasets/Train_Stations.xls")
geospatial_train_gdf = gpd.read_file(geospatial_train_path)

In [40]:
%run get_bus_info_function.ipynb
bus_services_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_routes_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_stops_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)

In [41]:
bus_routes_gdf = gpd.read_file('../datasets/filtered_bus_routes.geojson')

#### Data Pre-processing

In [42]:
bus_routes_stops = pd.merge(bus_routes_df, bus_stops_df, on = "BusStopCode", how = 'left')
bus_routes_stops = bus_routes_stops.merge(
    bus_services_df[['ServiceNo', 'Category']],  # Select only the columns needed for merging
    on='ServiceNo',  # Merge on BusStopCode
    how='left'  # Use 'left' join to keep all rows from bus_routes_stops
)

In [43]:
# Drop duplicates and assign it back to the original DataFrame
bus_routes_stops = bus_routes_stops.drop_duplicates().reset_index(drop=True)

# Filter the DataFrame for rows with 'Category' equal to 'TRUNK'
bus_routes_stops = bus_routes_stops[bus_routes_stops['Category'] == 'TRUNK']

In [44]:
# Step 1: Union the geometries for the same station
unioned_gdf = geospatial_train_gdf.dissolve(by='STN_NAM_DE',aggfunc='first')

# Step 2: Calculate the centroid of the unioned polygon
unioned_gdf['centroid'] = unioned_gdf.centroid
unioned_gdf['geometry'] = unioned_gdf['centroid']

# Reset index to clean up
unioned_gdf.reset_index(inplace=True)

In [45]:
# Function to normalize station names in train_stations_df
def normalize_station_name(name):
    return name.strip().upper()  # Ensure names are uppercase for consistent merging

# Apply normalization function to train_stations_df
train_stations['Normalized_Station'] = train_stations['MRT_Station'].apply(normalize_station_name)

# Create a column to append " MRT STATION" or " LRT STATION" based on the MRT_Line
train_stations['Station_MRT_LRT'] = train_stations.apply(
    lambda row: f"{row['Normalized_Station']} MRT STATION" if "LRT" not in row['MRT_Line'] else f"{row['Normalized_Station']} LRT STATION",
    axis=1
)

# Apply normalization to geospatial_train_df
# Strip ' MRT STATION' and ' LRT STATION' and normalize to uppercase
unioned_gdf['Normalized_Station'] = unioned_gdf['STN_NAM_DE'].str.strip().str.upper()

# Perform the merge on 'Station_MRT_LRT' from train_stations and 'Normalized_Station' from unioned_gdf
merged_train_stations = train_stations.merge(
    unioned_gdf,
    how='left',
    left_on='Station_MRT_LRT',
    right_on='Normalized_Station'
)

# Keeping necessary columns
columns_to_keep = ['Station_Code', 'MRT_Station', 'MRT_Line', 'TYP_CD_DES', 'geometry']
merged_train_stations = merged_train_stations[columns_to_keep]

# Check the resulting column names and sample data
print(merged_train_stations.head())


  Station_Code    MRT_Station           MRT_Line TYP_CD_DES  \
0          NS1    Jurong East  North-South Line         MRT   
1          NS2    Bukit Batok  North-South Line         MRT   
2          NS3   Bukit Gombak  North-South Line         MRT   
3          NS4  Choa Chu Kang  North-South Line         MRT   
4          NS5        Yew Tee  North-South Line         MRT   

                      geometry  
0  POINT (17866.487 35045.184)  
1  POINT (18676.448 36790.872)  
2  POINT (18940.178 37860.706)  
3  POINT (18101.056 40790.989)  
4  POINT (18438.643 42159.628)  


In [46]:
#  Convert Pandas DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_train_stations, geometry='geometry')

#  Reproject the GeoDataFrame to EPSG:4326 (WGS 84 - latitude/longitude)
gdf_4326 = gdf.to_crs(epsg=4326)

# Extract Longitude and Latitude from the reprojected geometries
gdf_4326['Longitude'] = gdf_4326.geometry.x
gdf_4326['Latitude'] = gdf_4326.geometry.y

#  Convert back to a Pandas DataFrame (if you don't need the geometry anymore)
merged_train_stations = pd.DataFrame(gdf_4326)

# Removing redundant columns
columns_to_keep = ['Station_Code', 'MRT_Station', 'MRT_Line', 'Longitude', 'Latitude']
merged_train_stations = merged_train_stations[columns_to_keep]
merged_train_stations['Train_Line'] = merged_train_stations['Station_Code'].str.extract(r'([A-Za-z]+)')
merged_train_stations['Station_No'] = merged_train_stations['Station_Code'].str.extract(r'(\d+)').fillna(1).astype(int)
print(merged_train_stations.head())

  Station_Code    MRT_Station           MRT_Line   Longitude  Latitude  \
0          NS1    Jurong East  North-South Line   103.742263  1.333209   
1          NS2    Bukit Batok  North-South Line   103.749541  1.348997   
2          NS3   Bukit Gombak  North-South Line   103.751910  1.358672   
3          NS4  Choa Chu Kang  North-South Line   103.744369  1.385172   
4          NS5        Yew Tee  North-South Line   103.747402  1.397550   

  Train_Line  Station_No  
0         NS           1  
1         NS           2  
2         NS           3  
3         NS           4  
4         NS           5  


In [47]:
def duplicate_and_modify_station_code(df, original_code, new_codes):
    """
    Duplicates a row based on the original station code and replaces it 
    with multiple new station codes.
    
    Parameters:
    df (pd.DataFrame): The original dataframe.
    original_code (str): The Station_Code to find and duplicate.
    new_codes (list of str): The list of new Station_Codes to replace the original with.
    
    Returns:
    pd.DataFrame: Updated dataframe with the original code replaced by new codes.
    """
    # Select the row with the specified original code
    original_row = df[df['Station_Code'] == original_code]
    
    # Create modified copies of the row for each new code
    new_rows = []
    for code in new_codes:
        new_row = original_row.copy()
        new_row['Station_Code'] = code
        new_rows.append(new_row)
    
    # Remove the original row and add the modified copies
    df = df[df['Station_Code'] != original_code]
    df = pd.concat([df] + new_rows, ignore_index=True)
    
    return df

In [48]:
# Step 1: Union the geometries for the same station
unioned_gdf = geospatial_train_gdf.dissolve(by='STN_NAM_DE',aggfunc='first')

# Step 2: Calculate the centroid of the unioned polygon
unioned_gdf['centroid'] = unioned_gdf.centroid

# Optional Step: Replace geometry with centroid point
unioned_gdf['geometry'] = unioned_gdf['centroid']

# Reset index to clean up
unioned_gdf.reset_index(inplace=True)

# Function to normalize station names in train_stations_df
def normalize_station_name(name):
    return name.strip().upper()  # Ensure names are uppercase for consistent merging

# Apply normalization function to train_stations_df
train_stations['Normalized_Station'] = train_stations['MRT_Station'].apply(normalize_station_name)

# Create a column to append " MRT STATION" or " LRT STATION" based on the MRT_Line
train_stations['Station_MRT_LRT'] = train_stations.apply(
    lambda row: f"{row['Normalized_Station']} MRT STATION" if "LRT" not in row['MRT_Line'] else f"{row['Normalized_Station']} LRT STATION",
    axis=1
)

# Apply normalization to geospatial_train_df
# Strip ' MRT STATION' and ' LRT STATION' and normalize to uppercase
unioned_gdf['Normalized_Station'] = unioned_gdf['STN_NAM_DE'].str.strip().str.upper()

# Perform the merge on 'Station_MRT_LRT' from train_stations and 'Normalized_Station' from unioned_gdf
merged_train_stations = train_stations.merge(
    unioned_gdf,
    how='left',
    left_on='Station_MRT_LRT',
    right_on='Normalized_Station'
)

merged_train_stations = merged_train_stations[['Station_Code', 'MRT_Station', 'MRT_Line', 'TYP_CD_DES', 'geometry']]

#  Convert Pandas DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_train_stations, geometry='geometry')

#  Reproject the GeoDataFrame to EPSG:4326 (WGS 84 - latitude/longitude)
gdf_4326 = gdf.to_crs(epsg=4326)

# Extract Longitude and Latitude from the reprojected geometries
gdf_4326['Longitude'] = gdf_4326.geometry.x
gdf_4326['Latitude'] = gdf_4326.geometry.y

#  Convert back to a Pandas DataFrame (if you don't need the geometry anymore)
geospatial_train_station = pd.DataFrame(gdf_4326)

geospatial_train_station = geospatial_train_station[['Station_Code', 'MRT_Station', 'MRT_Line', 'Longitude', 'Latitude']]

# Duplicate the row with 'PTC' and replace it with 'PW0', PW8', 'PE0', 'PE8'
geospatial_train_station = duplicate_and_modify_station_code(geospatial_train_station, 'PTC', ['PW0', 'PW8', 'PE0', 'PE8'])
# Duplicate the row with 'STC' and replace it with 'SW0', 'SW9', 'SE0', 'SE6'
geospatial_train_station = duplicate_and_modify_station_code(geospatial_train_station, 'STC', ['SW0', 'SW9', 'SE0', 'SE6'])
# Duplicate the row with 'BP6' and replace it with 'BP6', 'BP14'
geospatial_train_station = duplicate_and_modify_station_code(geospatial_train_station, 'BP6', ['BP6', 'BP14'])

geospatial_train_station['Train_Line'] = geospatial_train_station['Station_Code'].str.extract(r'([A-Za-z]+)')
geospatial_train_station['Station_No'] = geospatial_train_station['Station_Code'].str.extract(r'(\d+)').fillna(1).astype(int)

geospatial_train_station.head()

,Station_Code,MRT_Station,MRT_Line,Longitude,Latitude,Train_Line,Station_No
0,NS1,Jurong East,North-South Line,103.742263,1.333209,NS,1
1,NS2,Bukit Batok,North-South Line,103.749541,1.348997,NS,2
2,NS3,Bukit Gombak,North-South Line,103.751910,1.358672,NS,3
3,NS4,Choa Chu Kang,North-South Line,103.744369,1.385172,NS,4
4,NS5,Yew Tee,North-South Line,103.747402,1.397550,NS,5


In [49]:
# Convert to GeoDataFrame
train_stations_gdf = gpd.GeoDataFrame(
    geospatial_train_station,
    geometry=gpd.points_from_xy(geospatial_train_station.Longitude, geospatial_train_station.Latitude),
    crs="EPSG:4326"
)

# Step 2: Sort and group by train line to form continuous line segments for each line
train_stations_gdf = train_stations_gdf.sort_values(by=['Train_Line', 'Station_No'])

# Group by each train line to create LineString for each line
train_lines_gdf = train_stations_gdf.groupby('Train_Line').apply(
    lambda group: LineString(group.geometry.tolist()) if len(group) > 1 else None
).reset_index(name='geometry')

# Filter out rows where geometry is None (i.e., groups with less than 2 geometries)
train_lines_gdf = train_lines_gdf[train_lines_gdf['geometry'].notna()]

# Convert the result into a GeoDataFrame, which represents each train line as a LineString
train_lines_gdf = gpd.GeoDataFrame(train_lines_gdf, geometry='geometry', crs="EPSG:4326")

# Display the first few rows to confirm
print(train_lines_gdf.head())

  Train_Line                                           geometry
0         BP  LINESTRING (103.74455 1.38482, 103.74529 1.380...
1         CC  LINESTRING (103.84572 1.29938, 103.85066 1.296...
2         CE   LINESTRING (103.85908 1.28187, 103.85498 1.2757)
3         CG  LINESTRING (103.96205 1.33497, 103.98837 1.35731)
4         DT  LINESTRING (103.76157 1.37916, 103.7647 1.3693...


C:\Users\aiko\AppData\Local\Temp\ipykernel_38700\3452923685.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_lines_gdf = train_stations_gdf.groupby('Train_Line').apply(


In [50]:
# Convert bus stops (Pandas DataFrame) to GeoDataFrame with geometry points
bus_routes_stops_gdf = gpd.GeoDataFrame(
    bus_routes_stops,
    geometry=gpd.points_from_xy(bus_routes_stops['Longitude'], bus_routes_stops['Latitude']),
    crs="EPSG:4326"
)

### Method 

In [56]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import math
import pandas as pd
import numpy as np

# Constants
DISTANCE_THRESHOLD = 500  # Distance in meters
ANGLE_THRESHOLD = 50  # Maximum angle in degrees for coverage
projected_crs = "EPSG:3414"  # Change this to an appropriate projected CRS for your area

# Convert bus stops and train stations to GeoDataFrames with EPSG:4326
bus_routes_stops_gdf = gpd.GeoDataFrame(
    bus_routes_stops,
    geometry=gpd.points_from_xy(bus_routes_stops['Longitude'], bus_routes_stops['Latitude']),
    crs="EPSG:4326"
)
train_stations_gdf = gpd.GeoDataFrame(
    geospatial_train_station,
    geometry=gpd.points_from_xy(geospatial_train_station.Longitude, geospatial_train_station.Latitude),
    crs="EPSG:4326"
)

# Convert to projected CRS for accurate distance calculations
bus_routes_stops_gdf = bus_routes_stops_gdf.to_crs(projected_crs)
train_stations_gdf = train_stations_gdf.to_crs(projected_crs)

# Sort and group train stations to create continuous line segments for each train line
train_stations_gdf = train_stations_gdf.sort_values(by=['Train_Line', 'Station_No'])
train_lines_gdf = train_stations_gdf.groupby('Train_Line').apply(
    lambda group: LineString(group.geometry.tolist()) if len(group) > 1 else None
).reset_index(name='geometry')
train_lines_gdf = train_lines_gdf[train_lines_gdf['geometry'].notna()]
train_lines_gdf = gpd.GeoDataFrame(train_lines_gdf, geometry='geometry', crs=projected_crs)

# Function to calculate the angle between two points
def calculate_angle(bus_p1, bus_p2, train_p1, train_p2):
    bus_dx = bus_p1.x - bus_p2.x
    bus_dy = bus_p1.y - bus_p2.y
    bus_norm = math.sqrt(bus_dy ** 2 + bus_dx ** 2)

    train_dx = train_p1.x - train_p2.x
    train_dy = train_p1.y - train_p2.y
    train_norm = math.sqrt(train_dy ** 2 + train_dx ** 2)

    if bus_norm == 0 or train_norm == 0:
        return 180  # No angle if there is no valid segment

    norm_dot_prod = (bus_dx * train_dx + bus_dy * train_dy) / (bus_norm * train_norm)
    angle = np.degrees(np.arccos(norm_dot_prod))

    return angle if angle <= 90 else 180 - angle

# Function to find bus stops not covered by train stations and calculate a normalized score
def find_bus_stops_not_covered_by_trains(bus_routes_stops_gdf, train_stations_gdf, distance_threshold, angle_threshold):
    results = []
    for service_no, service_data in bus_routes_stops_gdf.groupby('ServiceNo'):
        total_stops = len(service_data)
        uncovered_count = 0
        for index, bus_stop in service_data.iterrows():
            bus_stop_geom = bus_stop.geometry
            nearby_train_stations = train_stations_gdf[train_stations_gdf.geometry.distance(bus_stop_geom) <= distance_threshold]
            if nearby_train_stations.empty:
                uncovered_count += 1
            else:
                for _, train_station in nearby_train_stations.iterrows():
                    train_station_geom = train_station.geometry
                    if index > 0 and index < total_stops - 1:
                        prev_bus_stop = service_data.iloc[index - 1].geometry
                        next_bus_stop = service_data.iloc[index + 1].geometry
                        bus_angle = calculate_angle(prev_bus_stop, next_bus_stop, train_station_geom, bus_stop_geom)
                        if bus_angle > angle_threshold:
                            uncovered_count += 1
        if total_stops > 0:
            proportion_uncovered = uncovered_count / total_stops
            score = proportion_uncovered
        else:
            weighted_score = 0
        results.append({
            'ServiceNo': service_no,
            'TotalStops': total_stops,
            'UncoveredStops': uncovered_count,
            'Score': score
        })
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='Score', ascending=False)
    return results_df

# Run the function to find uncovered bus stops and their scores
uncovered_bus_stops_df = find_bus_stops_not_covered_by_trains(bus_routes_stops_gdf, train_stations_gdf, DISTANCE_THRESHOLD, ANGLE_THRESHOLD)



C:\Users\aiko\AppData\Local\Temp\ipykernel_38700\3228668025.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_lines_gdf = train_stations_gdf.groupby('Train_Line').apply(


In [57]:
print(uncovered_bus_stops_df.head(20))

    ServiceNo  TotalStops  UncoveredStops     Score
241         6          30              28  0.933333
389       98B          27              25  0.925926
388       98A          11              10  0.909091
44        127          22              20  0.909091
340       927          33              30  0.909091
255        68          42              38  0.904762
127       17A          10               9  0.900000
29       117A          10               9  0.900000
331       89A          37              33  0.891892
132      181M          18              16  0.888889
262       70B          18              16  0.888889
64       139A           9               8  0.888889
131       181          18              16  0.888889
210       405          35              31  0.885714
377      975B          33              29  0.878788
15       109A          15              13  0.866667
257       68B           7               6  0.857143
256       68A           7               6  0.857143
158       19

In [58]:
# Filter the final results to exclude ServiceNos with alphabetic characters
filtered_final_results = uncovered_bus_stops_df[~uncovered_bus_stops_df['ServiceNo'].str.contains(r'[A-Za-z]')]

# Display filtered results
print(filtered_final_results[['ServiceNo', 'TotalStops','Score']].head(20))

    ServiceNo  TotalStops     Score
241         6          30  0.933333
44        127          22  0.909091
340       927          33  0.909091
255        68          42  0.904762
131       181          18  0.888889
210       405          35  0.885714
158       199          27  0.851852
112        17          62  0.838710
265        72          89  0.831461
194        35          35  0.828571
397       992          29  0.827586
180        29          54  0.814815
381        98          69  0.811594
133       182          58  0.810345
337       925          78  0.807692
144        19          56  0.803571
7         103          85  0.800000
0          10         148  0.797297
135       183          58  0.793103
163       201          38  0.789474


In [59]:
filtered_final_results.to_csv('../datasets/areas_without_train.csv')

## Optimal threshold

In [55]:
import geopandas as gpd
import numpy as np
import pandas as pd
import random
from joblib import Parallel, delayed

# Constants for threshold ranges
DISTANCE_THRESHOLD_RANGE = range(50, 501, 50)  # Distance from 50 to 500 meters
ANGLE_THRESHOLD_RANGE = range(0, 91, 5)  # Angle from 0 to 90 degrees in increments of 5
NUM_SAMPLES = 10  # Number of random samples to test for thresholds

# Randomly sample thresholds
sampled_thresholds = [
    (random.choice(DISTANCE_THRESHOLD_RANGE), random.choice(ANGLE_THRESHOLD_RANGE))
    for _ in range(NUM_SAMPLES)
]

# Define the function to evaluate thresholds
def evaluate_thresholds(distance_threshold, angle_threshold):
    uncovered_bus_stops_df = find_bus_stops_not_covered_by_trains(bus_routes_stops_gdf, train_stations_gdf, distance_threshold, angle_threshold)
    best_score = uncovered_bus_stops_df['Score'].mean()  # Adjust based on your criteria
    return distance_threshold, angle_threshold, best_score

# Use parallel processing to evaluate thresholds
results = Parallel(n_jobs=-1)(delayed(evaluate_thresholds)(dt, at) for dt, at in sampled_thresholds)

# Find the best combination from the results
best_combination = min(results, key=lambda x: x[2])  # Minimize the score
print("Optimal Thresholds:")
print(f"Distance Threshold: {best_combination[0]} meters")
print(f"Angle Threshold: {best_combination[1]} degrees")
print(f"Best Score: {best_combination[2]}")


Optimal Thresholds:
Distance Threshold: 500 meters
Angle Threshold: 50 degrees
Best Score: 0.5344018248091917
